### Predicting the Severity of Automobile Accidents in Seattle, Washington ###

In this first week, you will discover your
project objectives, find your dataset that you will use for this capstone project, and publish your
dataset on GitHub.

In the second week, you will build your machine
learning solution.

In the third week,
you will finalize your model and be ready
to submit your work.

To complete capstone,
you will be working on a case study which is to predict the severity
of an accident.
Now, wouldn't it be great if there were something in place that could warn you, 
given the weather and the road conditions,
about the possibility of you getting into a car accident and how severe it would be,
so that you would drive more carefully or even change your travel plans?
Let's use our shared data for Seattle, Washington as an example of how to deal with the accidents data.

In [15]:
# Load the pandas package.
import pandas as pd

# Load the numpy package.
import numpy as np

In [16]:
# Read the Collisions Data CSV file and store it as a DataFrame.
url = "https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv"
df_collisions = pd.read_csv(url, low_memory=False)

In [17]:
# Print the first five rows of the DataFrame.
df_collisions.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [18]:
# Print the data type of each column of the DataFrame.
df_collisions.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [19]:
df_collisions.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [20]:
# Drop columns from the DataFrame that contain identifiers/keys with have no predictive value.
id_columns = ['OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'EXCEPTRSNCODE', 'SDOTCOLNUM', ]
df_collisions.drop(columns=id_columns, inplace=True)
df_collisions['ST_COLCODE'].value_counts().head(50)

32    44421
10    34674
14    25771
50    13554
11    12777
28    10324
13     7629
       4886
45     4701
23     4537
15     3093
16     2956
0      2882
20     2846
12     2435
22     2274
2      2178
21     1617
30     1302
1      1201
71     1184
26     1039
81      835
52      815
19      720
24      590
5       416
51      371
74      343
29      286
6       257
3       236
27      231
73      167
25      132
4       111
57      108
40      103
84       94
83       86
72       73
41       57
64       50
31       47
82       35
56       34
48       32
53       26
8        23
7        18
Name: ST_COLCODE, dtype: int64

In [7]:
# Compare two columns of DataFrame that have similar names.
pd.Series(df_collisions["SEVERITYCODE"] == df_collisions["SEVERITYCODE.1"]).value_counts()

True    194673
dtype: int64

In [8]:
# Because columns SEVERITYCODE and SEVERITYCODE.1 are identical, drop column SEVERITYCODE.1 from the DataFrame.
df_collisions.drop(columns="SEVERITYCODE.1", inplace=True)

In [9]:
# Print a concise, technical summary of the DataFrame.
df_collisions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 31 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEVERITYCODE    194673 non-null  int64  
 1   X               189339 non-null  float64
 2   Y               189339 non-null  float64
 3   STATUS          194673 non-null  object 
 4   ADDRTYPE        192747 non-null  object 
 5   INTKEY          65070 non-null   float64
 6   LOCATION        191996 non-null  object 
 7   EXCEPTRSNDESC   5638 non-null    object 
 8   SEVERITYDESC    194673 non-null  object 
 9   COLLISIONTYPE   189769 non-null  object 
 10  PERSONCOUNT     194673 non-null  int64  
 11  PEDCOUNT        194673 non-null  int64  
 12  PEDCYLCOUNT     194673 non-null  int64  
 13  VEHCOUNT        194673 non-null  int64  
 14  INCDATE         194673 non-null  object 
 15  INCDTTM         194673 non-null  object 
 16  JUNCTIONTYPE    188344 non-null  object 
 17  SDOT_COLCO

In [10]:
# Print the statistics for each numerical column of the DataFrame.
df_collisions.describe()

,SEVERITYCODE,X,Y,INTKEY,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SEGLANEKEY,CROSSWALKKEY
count,194673.000000,189339.000000,189339.000000,65070.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,1.946730e+05
mean,1.298901,-122.330518,47.619543,37558.450576,2.444427,0.037139,0.028391,1.920780,13.867768,269.401114,9.782452e+03
std,0.457778,0.029976,0.056157,51745.990273,1.345929,0.198150,0.167413,0.631047,6.868755,3315.776055,7.226926e+04
min,1.000000,-122.419091,47.495573,23807.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,1.000000,-122.348673,47.575956,28667.000000,2.000000,0.000000,0.000000,2.000000,11.000000,0.000000,0.000000e+00
50%,1.000000,-122.330224,47.615369,29973.000000,2.000000,0.000000,0.000000,2.000000,13.000000,0.000000,0.000000e+00
75%,2.000000,-122.311937,47.663664,33973.000000,3.000000,0.000000,0.000000,2.000000,14.000000,0.000000,0.000000e+00
max,2.000000,-122.238949,47.734142,757580.000000,81.000000,6.000000,2.000000,12.000000,69.000000,525241.000000,5.239700e+06


In [11]:
# Identify missing data in the DataFrame.
missing_data = df_collisions.isnull()

print("True corresponds to NaN in the Collisions DataFrame.")
print()
# Count missing values in each column
for column in missing_data.columns.values.tolist():
    print(column, ":", sep='')
    print("Absolute frequencies of the unique values (not normalized):")
    print (missing_data[column].value_counts())
    print()
    print("Relative frequencies of the unique values (normalized)")
    print (missing_data[column].value_counts(normalize=True))
    print()    

True corresponds to NaN in the Collisions DataFrame.

SEVERITYCODE:
Absolute frequencies of the unique values (not normalized):
False    194673
Name: SEVERITYCODE, dtype: int64

Relative frequencies of the unique values (normalized)
False    1.0
Name: SEVERITYCODE, dtype: float64

X:
Absolute frequencies of the unique values (not normalized):
False    189339
True       5334
Name: X, dtype: int64

Relative frequencies of the unique values (normalized)
False    0.9726
True     0.0274
Name: X, dtype: float64

Y:
Absolute frequencies of the unique values (not normalized):
False    189339
True       5334
Name: Y, dtype: int64

Relative frequencies of the unique values (normalized)
False    0.9726
True     0.0274
Name: Y, dtype: float64

STATUS:
Absolute frequencies of the unique values (not normalized):
False    194673
Name: STATUS, dtype: int64

Relative frequencies of the unique values (normalized)
False    1.0
Name: STATUS, dtype: float64

ADDRTYPE:
Absolute frequencies of the unique val

In [12]:
# Deal with missing data.

In [13]:
# Correct data format.
